# Image Modelling Part 2 - Use Pipeline

In the first notebook we used shell commands to prepare and split our data into a train and evaluation set. 
Furthermore, we defined some functions that will allow us to directly import our pictures and the corresponding class labels and if we want to also augment our data. 
Now, we will import the functions from the `image_modelling.py` file and use them to facilitate the data preparation step in this notebook. 
Lastly, we will use Tensorflow and Keras to create and train our neuronal network to identify turtles.

In [ ]:
# Import required packages 
import tensorflow as tf
import image_modeling   # import image_modeling.py file
import tensorflow_hub as hub
import datetime
import csv
import numpy as np
import pandas as pd
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
# Check for Tensorflow version
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

In [ ]:
# Import variables from image_modelling.py file
file = open("../data/train.csv")
reader = csv.reader(file)

HEIGHT = image_modeling.HEIGHT
WIDTH = image_modeling.WIDTH
NCLASSES = image_modeling.NCLASSES
CLASS_NAMES = image_modeling.CLASS_NAMES
BATCH_SIZE = image_modeling.BATCH_SIZE
TRAINING_SIZE = image_modeling.TRAINING_SIZE
TRAINING_STEPS = (TRAINING_SIZE // BATCH_SIZE)

Double check if the variables now contain the correct values. ;) 

In [ ]:
# You can compare this output with the variables in the image_modelling.py file...
print(HEIGHT)
print(CLASS_NAMES)
print(NCLASSES)
print(TRAINING_STEPS)
print(TRAINING_SIZE)

## Building our Model

Building and training a neural network involves various steps: 
1. define the architecture of the model
2. compile the model
3. train the model
4. evaluate the model

We have to start with defining the architecture. Our neural network will consist of several layers that are chained together. The input layer of our model will take our input data and hand it over to the flatten layer, which is responsible for reformatting our data. It will transform the format of our images from a three-dimensional array (HEIGHT, WIDTH, 3) to a one-dimensional array of size HEIGHT * WIDTH * 3. 
After the pixels are flattened we use a dense layer that returns a logits array with length `NCLASSES`. Each node in this layer contains a score that indicates the current image belongs to one of the n classes. 

### Simple Model

In [ ]:
'''
# Lets create a simple linear model.
def linear_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Flatten(data_format="channels_last"))
    # We want to have a simple linear model so we have 
    # no activation function. 
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model
    '''

Before we can train our model we need to compile it and define more settings. We have to choose a loss function, an optimizer and metrics. 
* The **loss function** measures how accurate the model is during training by calculating the model error. Usually we want to minimize this function to improve our model. As you can see in the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/losses) there are lot's of different loss functions to choose from. Some, e.g. the mean squared errror, hopefully look familiar to you. ;) 
* The **[optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)** defines how the model is updated based on the data and the loss function. One optimizer we've already covered earlier and which is also used for neural networks is the stochastic gradient descent (SGD) algorithm.   
* The **metric** is used to monitor the training process. Here we can choose one of the metrics we've already encountered or [many more](https://www.tensorflow.org/api_docs/python/tf/keras/metrics). 

The following function compiles our model, loads the data using the `load_dataset()` function from the image_modelling.py file and trains the model on the loaded data. In the end the function returns our fitted model. 

In [ ]:
'''
def train_and_evaluate(model,batch_size=32):

    model.compile(
        optimizer="adam", 
        # The model outputs one-hot-encoded logits, so we need
        # use the sparse version of the crossentropy loss.
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    train_datagen, test_datagen = image_modeling.preprocess()
    train_generator, validation_generator = image_modeling.use_image_generator(train_datagen, test_datagen, training=True)
    
    model.fit(
        train_generator, 
        validation_data=validation_generator,
        steps_per_epoch=TRAINING_STEPS, 
        epochs=10,
        callbacks=[tensorboard_callback])
          
    return model
    '''

In [ ]:
''' # Build and train our model using the prior defined functions 
model = linear_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)
'''

Let us use Tensorboard to monitor our results:

In [ ]:
'''
%tensorboard --logdir logs/fit
'''

### Deep Neural Network

Our simple model is not performing well. Maybe we can boost its performance by adding more layers.

In the following `dnn_model()` function we add three more hidden, dense layers after the flatten layer to increase our models complexity. 

In [ ]:
'''
# Lets compare a neural network with hidden layers to the linear model
def dnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Flatten(data_format="channels_last"))
    model.add(tf.keras.layers.Dense(units = 40, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 40, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 30, activation = "relu"))
    # We want to have a simple linear model so we have 
    # no activation function. 
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model
    '''

In [ ]:
'''
# Let us fit the deep neural network
model = dnn_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)
'''

Adding more hidden layers to our model, did indeed increase the accuracy. But still, the model's performance leaves something to be desired. Since we are working with images, switching to a convolutional neural network might help.

### Convolutional Neural Network

CNN's are widely used for image recognition. They are regularized versions of DNN's able to be deeper without generating as much parameters due to its [convolutional](https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/) and pooling layers.

The architecture of our CNN is even more complex. This time we combine dense layers with `Conv2D` and `MaxPooling2D` layers. The convolutional and max pooling layers are inserted between the input layer and the flatten layer. 

In [ ]:
'''
# Now let us move on to a CNN model. 
def cnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation="relu"))
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model
    '''

We can have a look at the architecture of our model with the method `.summary`. As you can see in the summary below, the output of each `Conv2D` and `MaxPooling2D` layer is also a three dimensional tensor of shape (height, width, channels). As we go deeper into the network the dimensions shrink. One advantage of the shrinking dimensions is that we can computationally afford to add more output channels in each convolutional layer. We can control the number of the output channels of those layers with the `filters` argument. 

However, at the end of our model we still need the combination of the flatten and dense layers to perform classification. 

In [ ]:
'''
model = cnn_model()
model.summary()
'''

In [ ]:
'''
# Let us fit the convolutional neural network
model = cnn_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)
'''

We see the CNN give better results than the DNN. But we still have heavy overfitting.

## Transfer Learning

Transfer learning is when a model is trained on one task and is then reused for another task. One approach to transfer learning is fine-tuning. Here you take a trained neural net, exchange the last layer (head) for another layer, that fits the new task and then train the weights of the last layer only. 

First we need to download the headless model (this can take a while) we use MobileNetV2 which is a CNN that was trained on the [ImageNet](https://en.wikipedia.org/wiki/ImageNet) dataset, consisting of over 14 million images:

In [ ]:
'''
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,input_shape=(HEIGHT,WIDTH,3))
'''

We only want to train the last layer therefore we freeze the layers of our headless model:

In [ ]:
'''
feature_extractor_layer.trainable = False
'''

Now we define our model by simply adding the output layer to our pretrained net.

In [ ]:
'''
def transfer_learning_model():
    model = tf.keras.models.Sequential()
    model.add(feature_extractor_layer)
    # TODO: add the correct output layer here
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation="relu"))
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model
    '''

In [ ]:
'''
# Let us fit our transfer learning model
model = transfer_learning_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)
'''

In [ ]:
'''
%tensorboard --logdir logs/fit
'''

As we see the results of fine-tuning surpass the results of the linear model, DNN and CNN. Fine-tuning is a very powerful approach which can generalize well even with limited amount of data.

### Transfer Learning InceptionV3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets, layers, models

In [ ]:
#change the last layer
for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final softmax layer with 101 nodes for classification output
x = layers.Dense(NCLASSES, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = image_modeling.preprocess()
train_generator, validation_generator = image_modeling.use_image_generator(train_datagen, test_datagen, training=True)
    
inception =  model.fit(
        train_generator, 
        validation_data=validation_generator,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=10,
        callbacks=[tensorboard_callback])

In [ ]:
#%tensorboard --logdir logs/fit

### Concatinated Model with location

In [ ]:
'''
train_data_loc = image_modeling.location_encoding(image_modeling.train_data)
train_loc = train_data_loc[0:image_modeling.lines][['image_location_left', 'image_location_right', 'image_location_top']]
val_loc = train_data_loc[image_modeling.lines:][['image_location_left', 'image_location_right', 'image_location_top']]
train_loc.head()
'''

In [ ]:
'''
# Image model
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets, layers, models

#InceptionV3 layers

#change the last layer
for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

img_model = tf.keras.models.Model(base_model.input, outputs=x)
'''

In [ ]:
'''
# Location model
loc_input = layers.Input(shape=(image_modeling.lines,))
y = layers.Dense(1024, activation='relu')(loc_input)
loc_model = tf.keras.models.Model(loc_input, outputs=y)
'''

In [ ]:
'''
# Combined model
# combine the output of the two branches
combined = layers.concatenate([img_model.output, loc_model.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
# Add a final softmax layer with 101 nodes for classification output
z = layers.Dense(NCLASSES, activation='softmax')(combined)
# our model will accept the inputs of the two branches and
# then output a single value
model = tf.keras.models.Model(inputs=[img_model.input, loc_model.input], outputs=z)
'''

In [ ]:
'''
# NOT WORKING!!!


model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = image_modeling.preprocess()
train_generator, validation_generator = image_modeling.use_image_generator(train_datagen, test_datagen, training=True)
    
inception =  model.fit(
        x=[train_generator[0], train_loc], y=train_generator[1],
        validation_data=([validation_generator[0], val_loc],validation_generator[1]),
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=10,
        callbacks=[tensorboard_callback])
'''

### Concatenate Model with partial model for each location

In [ ]:
# Image model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets, layers, models

'''
#InceptionV3 layers
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
#change the last layer
for layer in base_model.layers:
    layer.trainable = False
'''

In [ ]:
#InceptionV3 layers
base_model_left = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
#change the last layer
for layer in base_model_left.layers:
    layer.trainable = False
    layer._name = layer.name + str("_left")
    
#new layers for left image location
left = layers.Flatten()(base_model_left.output)
left = layers.Dense(1024, activation='relu')(left)
left = layers.Dropout(0.2)(left)

left = tf.keras.models.Model(base_model_left.input, outputs=left, name='left')

In [ ]:
#InceptionV3 layers
base_model_right = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
#change the last layer
for layer in base_model_right.layers:
    layer.trainable = False
    layer._name = layer.name + str("_right")

#new layers for right image location
right = layers.Flatten()(base_model_right.output)
right = layers.Dense(1024, activation='relu')(right)
right = layers.Dropout(0.2)(right)

right = tf.keras.models.Model(base_model_right.input, outputs=right, name='right')

In [ ]:
#InceptionV3 layers
base_model_top = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
#change the last layer
for layer in base_model_top.layers:
    layer.trainable = False
    layer._name = layer.name + str("_top")

#new layers for right image location
top = layers.Flatten()(base_model_top.output)
top = layers.Dense(1024, activation='relu')(top)
top = layers.Dropout(0.2)(top)

top = tf.keras.models.Model(base_model_top.input, outputs=top, name='top')

In [ ]:
# Combined model
# combine the output of the two branches
combined = layers.concatenate([left.output, right.output, top.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
# Add a final softmax layer with 101 nodes for classification output
z = layers.Dense(NCLASSES, activation='softmax')(combined)
# our model will accept the inputs of the two branches and
# then output a single value
combined_model = tf.keras.models.Model(inputs=[left.input, right.input, top.input], outputs=z)

In [ ]:
combined_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#ImageDataGenerator
train_datagen, test_datagen = image_modeling.preprocess()
train_generator_left, validation_generator_left = image_modeling.use_image_generator(image_modeling.train_data_left, train_datagen, test_datagen, training=True)
train_generator_right, validation_generator_right = image_modeling.use_image_generator(image_modeling.train_data_right, train_datagen, test_datagen, training=True)
train_generator_top, validation_generator_top = image_modeling.use_image_generator(image_modeling.train_data_top, train_datagen, test_datagen, training=True)
    
inception =  combined_model.fit(
        x=[train_generator_left[0][0], train_generator_right[0][0], train_generator_top[0][0]],
        y=[train_generator_left[0][1], train_generator_right[0][1], train_generator_top[0][1]],
        validation_data=([validation_generator_left[0][0], validation_generator_right[0][0], validation_generator_top[0][0]],
                         [validation_generator_left[0][1], validation_generator_right[0][1], validation_generator_top[0][1]]),
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=10,
        callbacks=[tensorboard_callback])

In [ ]:
train_generator_left[0][0].shape

In [ ]:
def add_prefix(model, prefix: str, custom_objects=None):
    '''https://nrasadi.medium.com/change-model-layer-name-in-tensorflow-keras-58771dd6bf1b
    Adds a prefix to layers and model name while keeping the pre-trained weights
    Arguments:
        model: a tf.keras model
        prefix: a string that would be added to before each layer name
        custom_objects: if your model consists of custom layers you shoud add them pass them as a dictionary. 
            For more information read the following:
            https://keras.io/guides/serialization_and_saving/#custom-objects
    Returns:
        new_model: a tf.keras model having same weights as the input model.
    '''
    
    config = model.get_config()
    old_to_new = {}
    new_to_old = {}
    
    for layer in config['layers']:
        new_name = prefix + layer['name']
        old_to_new[layer['name']], new_to_old[new_name] = new_name, layer['name']
        layer['name'] = new_name
        layer['config']['name'] = new_name

        if len(layer['inbound_nodes']) > 0:
            for in_node in layer['inbound_nodes'][0]:
                in_node[0] = old_to_new[in_node[0]]
    
    for input_layer in config['input_layers']:
        input_layer[0] = old_to_new[input_layer[0]]
    
    for output_layer in config['output_layers']:
        output_layer[0] = old_to_new[output_layer[0]]
    
    config['name'] = prefix + config['name']
    new_model = tf.keras.Model().from_config(config, custom_objects)
    
    for layer in new_model.layers:
        layer.set_weights(model.get_layer(new_to_old[layer.name]).get_weights())
    
    return new_model

### 3 different models for location

In [ ]:
# Image model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets, layers, models

In [ ]:
#InceptionV3 layers
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

#change the last layer
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#new layers for left image location
left = layers.Flatten()(base_model.output)
left = layers.Dense(1024, activation='relu')(left)
left = layers.Dropout(0.2)(left)

# Add a final softmax layer with 101 nodes for classification output
left = layers.Dense(NCLASSES, activation='softmax')(left)

model_left = tf.keras.models.Model(base_model.input, left)

model_left.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = image_modeling.preprocess()
train_generator_left, validation_generator_left = image_modeling.use_image_generator(image_modeling.train_data_left, 
                                                                                     train_datagen, test_datagen, training=True)
    
inception_left =  model_left.fit(
        train_generator_left, 
        validation_data=validation_generator_left,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=20,
        callbacks=[tensorboard_callback])

In [ ]:
#new layers for left image location
right = layers.Flatten()(base_model.output)
right = layers.Dense(1024, activation='relu')(right)
right = layers.Dropout(0.2)(right)

# Add a final softmax layer with 101 nodes for classification output
right = layers.Dense(NCLASSES, activation='softmax')(right)

model_right = tf.keras.models.Model(base_model.input, right)

model_right.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = image_modeling.preprocess()
train_generator_right, validation_generator_right = image_modeling.use_image_generator(image_modeling.train_data_right, 
                                                                                     train_datagen, test_datagen, training=True)
    
inception_right =  model_right.fit(
        train_generator_right, 
        validation_data=validation_generator_right,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=20,
        callbacks=[tensorboard_callback])

In [ ]:
#new layers for left image location
top = layers.Flatten()(base_model.output)
top = layers.Dense(1024, activation='relu')(top)
top = layers.Dropout(0.2)(top)

# Add a final softmax layer with 101 nodes for classification output
top = layers.Dense(NCLASSES, activation='softmax')(top)

model_top = tf.keras.models.Model(base_model.input, top)

model_top.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = image_modeling.preprocess()
train_generator_top, validation_generator_top = image_modeling.use_image_generator(image_modeling.train_data_top, 
                                                                                     train_datagen, test_datagen, training=True)
    
inception_top =  model_top.fit(
        train_generator_top, 
        validation_data=validation_generator_top,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=20,
        callbacks=[tensorboard_callback])

## Prepare data for submission

In [ ]:
test_data = image_modeling.test_data
test_generator = image_modeling.use_image_generator(test_data, train_datagen, test_datagen, training=False)
#test_generator[489].shape

In [ ]:
y_preds = []
for index in range(len(test_data)):
    if test_data['image_location'][index] == 'left': 
        y_pred = model_left.predict(test_generator[index])
        y_preds.append(y_pred.flatten())
    elif test_data['image_location'][index] == 'right':
        y_pred = model_right.predict(test_generator[index])
        y_preds.append(y_pred.flatten())
    else:
        y_pred = model_top.predict(test_generator[index])
        y_preds.append(y_pred.flatten())

In [ ]:
#Get probabilities for all turtle id's
#y_preds = model.predict(test_generator)
print(y_preds[0])
#Get indices from top 5 predictions
# Corrected: [:,:-6:-1] instead of [:,-5:]
y_preds = np.argsort(y_preds, axis=1)[:,:-6:-1]

#Save indices of top 5 predictions as dataframe
df = pd.DataFrame(y_preds)

In [ ]:
df

In [ ]:
#Create a DataFrame with top 5 predictions in submission form
list = []
array = []
for line in y_preds:
    for id in line:
        list.append(CLASS_NAMES[id])
    array.append(list)
    list = []

titles = ['prediction1', 'prediction2','prediction3','prediction4','prediction5']

submission = pd.DataFrame(array, columns= titles)

#Insert image_ids from test_data
test_data = pd.read_csv('../data/test.csv')
submission.insert(loc=0, column='image_id', value=test_data['image_id'])
submission

In [ ]:
#Save submission data as CSV
submission.to_csv('../data/submission_shuffleFalse.csv', index = False)

### Misc

In [ ]:
#Save model
'''
model.save('InceptionV3')
'''

In [ ]:
#Load model
'''
loaded_model = tf.keras.models.load_model('InceptionV3)
loaded_model.layers[0].input_shape #(None, 150, 150, 3)
'''